**Exercicio 1:**<br>
Implemente a classe de arvores binarias LinkedBinaryTree usando uma estrutura encadeada conforme descrito na Secao 8.3.1

In [1]:
class Tree:
    class Position:
        def elements(self):
            raise NotImplementedError('Must be implemented by subclass.')
        def __eq__(self, other):
            raise NotImplementedError('Must be implemented by subclass.')
        def __ne__(self, other):
            return not (self == other)
    def root(self):
        raise NotImplementedError('Must be implemented by subclass.')    
    def parent(self, p):
        raise NotImplementedError('Must be implemented by subclass.')    
    def num_children(self, p):
        raise NotImplementedError('Must be implemented by subclass.')    
    def children(self, p):
        raise NotImplementedError('Must be implemented by subclass.')    
    def __len__(self):
        raise NotImplementedError('Must be implemented by subclass.')    
    def is_root(self, p):
        return self.root() == p
    def is_leaf(self, p):
        return self.num_children(p) == 0
    def is_empty(self):
        return len(self) == 0

class BinaryTree(Tree):
    def left(self, p):
        raise NotImplementedError('Must be implemented by subclass.')
    def right(self, p):
        raise NotImplementedError('Must be implemented by subclass.')    
    def sibling(self, p):
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
    def children(self, p):
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)


class LinkedBinaryTree(BinaryTree):
    class _Node:
        __slots__ = '_element', '_parent', '_left', '_right'
        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right
    
    class Position(BinaryTree.Position):
        def __init__(self, container, node):
            self._container = container
            self._node = node
        
        def element(self):
            return self._node._element
        
        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node
    
    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position Type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node:
            raise ValueError('p is no longer valid')
        return p._node
    
    def _make_position(self, node):
        return self.Position(self, node) if node is not None else None
    
    def __init__(self):
        self._root = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def root(self):
        return self._make_position(self._root)
    
    def parent(self, p):
        node = self._validate(p)
        return self._make_position(node._parent)

    def left(self, p):
        node = self._validate(p)
        return self._make_position(node._left)
    
    def right(self, p):
        node = self._validate(p)
        return self._make_position(node._right)
    
    def num_children(self, p):
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1
        if node._right is not None:
            count += 1
        return count
    
    def _add_root(self, e):
        if self._root is not None: raise ValueError('Root exists.')
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)
    
    def _add_left(self, p, e):
        node = self._validate(p)
        if node._left is not None: raise ValueError('Left child exists.')
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)
    
    def _add_right(self, p, e):
        node = self._validate(p)
        if node._right is not None: raise ValueError('Left child exists.')
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)
    
    def _replace(self, p, e):
        node = self._validate(p)
        old = node._element
        node._element = e
        return old
    
    def _delete(self, p):
        node = self._validate(p)
        if self.num_children(p) == 2: raise ValueError('p has two children.')
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent
        if node is self._root:
            self._root = child
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element
    
    def _attach(self, p, t1, t2):
        node = self._validate(p)
        if not self.is_leaf(p): raise ValueError('Position must be leaf.')
        if not type(self) is type(t1) is type(t2):
            raise TypeError('Tree types must match.')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root
            t2._root = None
            t2._size = 0
        

**Exercicio 2:**<br>
Implemente a classe de arvore binaria ArrayBinaryTree usando a representacao baseada em array descrita na Secao 8.3.2. Implemente as mesmas funcoes presentes na classe LinkedBinaryTree

In [2]:
class ArrayBinaryTree(BinaryTree):
    
    class _ArrayPosition(BinaryTree.Position):
        def __init__(self, container, index):
            self._container = container
            self._index = index
        
        def element(self):
            return self._container._data[self._index]
        
        def __eq__(self, other):
            return type(other) is type(self) and other._index == self._index
    
    def __init__(self, max_size=100):
        self._data = [None] * max_size
        self._size = 0
        self._max_size = max_size
    
    def __len__(self):
        return self._size
    
    def _validate(self, p):
        if not isinstance(p, self._ArrayPosition):
            raise TypeError('p must be proper Position type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._index >= self._max_size or self._data[p._index] is None:
            raise ValueError('p is no longer valid')
        return p._index
    
    def _make_position(self, index):
        if 0 <= index < self._max_size and self._data[index] is not None:
            return self._ArrayPosition(self, index)
        return None
    
    def root(self):
        return self._make_position(0)
    
    def parent(self, p):
        index = self._validate(p)
        if index == 0:
            return None
        parent_index = (index - 1) // 2
        return self._make_position(parent_index)
    
    def left(self, p):
        index = self._validate(p)
        left_index = 2 * index + 1
        return self._make_position(left_index)
    
    def right(self, p):
        index = self._validate(p)
        right_index = 2 * index + 2
        return self._make_position(right_index)
    
    def num_children(self, p):
        count = 0
        if self.left(p) is not None:
            count += 1
        if self.right(p) is not None:
            count += 1
        return count
    
    def add_root(self, e):
        if not self.is_empty():
            raise ValueError('Root exists')
        self._data[0] = e
        self._size = 1
        return self._make_position(0)
    
    def add_left(self, p, e):
        index = self._validate(p)
        left_index = 2 * index + 1
        if left_index >= self._max_size:
            raise ValueError('Tree is full')
        if self._data[left_index] is not None:
            raise ValueError('Left child exists')
        self._data[left_index] = e
        self._size += 1
        return self._make_position(left_index)
    
    def add_right(self, p, e):
        index = self._validate(p)
        right_index = 2 * index + 2
        if right_index >= self._max_size:
            raise ValueError('Tree is full')
        if self._data[right_index] is not None:
            raise ValueError('Right child exists')
        self._data[right_index] = e
        self._size += 1
        return self._make_position(right_index)
    
    def replace(self, p, e):
        index = self._validate(p)
        old = self._data[index]
        self._data[index] = e
        return old
    
    def _delete(self, p):
        index = self._validate(p)
        
        if self.num_children(p) == 2:
            raise ValueError('p has two children')
        
        child_pos = self.left(p) if self.left(p) else self.right(p)
        element_to_return = self._data[index]
        
        if child_pos is not None:
            child_index = child_pos._index
            self._shift_subtree_up(child_index, index)
        
        self._data[index] = None
        self._size -= 1
        return element_to_return

    def _shift_subtree_up(self, from_index, to_index):
        if from_index >= self._max_size or self._data[from_index] is None:
            return
            
        index_diff = to_index - from_index
        queue = [from_index]
        nodes_to_move = []

        while queue:
            current_from = queue.pop(0)
            if current_from < self._max_size and self._data[current_from] is not None:
                nodes_to_move.append(current_from)
                
                left_from = 2 * current_from + 1
                right_from = 2 * current_from + 2

                if left_from < self._max_size:
                    queue.append(left_from)
                if right_from < self._max_size:
                    queue.append(right_from)

        for current_from in nodes_to_move:
            current_to = current_from + index_diff
            if current_to < self._max_size:
                self._data[current_to] = self._data[current_from]
                self._data[current_from] = None

    def _attach(self, p, t1, t2):
        index = self._validate(p)
        if not self.is_leaf(p):
            raise ValueError('Position must be leaf')
        if not type(self) is type(t1) is type(t2):
            raise TypeError('Tree types must match')
        
        if not t1.is_empty():
            self._attach_subtree(t1, 0, index, True)
            t1._data = [None] * t1._max_size
            t1._size = 0
            
        if not t2.is_empty():
            self._attach_subtree(t2, 0, index, False)
            t2._data = [None] * t2._max_size
            t2._size = 0
    
    def _attach_subtree(self, source_tree, source_index, target_parent_index, is_left):
        if source_index >= source_tree._max_size or source_tree._data[source_index] is None:
            return
        
        target_index = 2 * target_parent_index + (1 if is_left else 2)
        
        if target_index >= self._max_size:
            raise ValueError('Not enough space to attach subtree')
        
        if self._data[target_index] is not None:
             raise ValueError('Target position already occupied')

        self._data[target_index] = source_tree._data[source_index]
        self._size += 1
        
        source_left = 2 * source_index + 1
        if source_left < source_tree._max_size:
            self._attach_subtree(source_tree, source_left, target_index, True)
            
        source_right = 2 * source_index + 2
        if source_right < source_tree._max_size:
            self._attach_subtree(source_tree, source_right, target_index, False)

if __name__ == '__main__':
    tree = ArrayBinaryTree(20)
    
    root = tree.add_root(10)
    left_child = tree.add_left(root, 5)
    right_child = tree.add_right(root, 15)
    
    leaf1 = tree.add_left(left_child, 3)
    leaf2 = tree.add_right(right_child, 18)
    
    print(f"Tree size: {len(tree)}")
    print(f"Root element: {root.element()}")
    
    deleted = tree._delete(leaf1)
    print(f"Deleted element: {deleted}")
    print(f"Tree size after delete: {len(tree)}")
    
    t1 = ArrayBinaryTree(20)
    t1_root = t1.add_root(20)
    t1.add_left(t1_root, 18)
    t1.add_right(t1_root, 22)
    
    t2 = ArrayBinaryTree(20)
    t2_root = t2.add_root(30)
    
    tree._attach(left_child, t1, t2)
    print(f"Tree size after attach: {len(tree)}")

Tree size: 5
Root element: 10
Deleted element: 3
Tree size after delete: 4
Tree size after attach: 8


**Exercicio 3:**<br>
Implemente as funcoes Postorder, Inorder e Preorder Traversal (secao 8.4.4) para a classe LinkedBinaryTree

In [3]:
# Estrutura:
#       1
#     /   \
#    2     3
#   / \   / \
#  4   5 6   7

tree = LinkedBinaryTree()
root = tree._add_root('1')      
n2 = tree._add_left(root, '2')      
n3 = tree._add_right(root, '3')  
n4 = tree._add_left(n2, '4')   
n5 = tree._add_right(n2, '5')  
n6 = tree._add_left(n3, '6')        
n7 = tree._add_right(n3, '7')        

# Implementing PREORDER TRAVERSAL method
def preorder(self):
    if not self.is_empty():
        for p in self._subtree_preoder(self.root()):
            yield p

def _subtree_preorder(self, p):
    yield p
    for c in self.children(p):
        for other in self._subtree_preoder(c):
            yield other

LinkedBinaryTree._subtree_preoder = _subtree_preorder
LinkedBinaryTree.preorder = preorder

# Expected output: 1, 2, 4, 5, 3, 6, 7
print('--PREORDER TRAVERSAL--')
for p in tree.preorder():
    print(p.element())
print("\n")

# Implementing POSTORDER TRAVERSAL method
def postorder(self):
    if not self.is_empty():
        for p in self._subtree_postorder(self.root()):
            yield p

def _subtree_postorder(self, p):
    for c in self.children(p):
        for other in self._subtree_postorder(c):
            yield other
    yield p

LinkedBinaryTree._subtree_postorder = _subtree_postorder
LinkedBinaryTree.postorder = postorder

# Expected output: 4, 5, 2, 6, 7, 3, 1
print('--POSTORDER TRAVERSAL--')
for p in tree.postorder():
    print(p.element())
print("\n")


# Implementing INORDER TRAVERSAL method
def inorder(self):
    if not self.is_empty():
        for p in self._subtree_inorder(self.root()):
            yield p

def _subtree_inorder(self, p):
    if self.left(p) is not None:
        for other in self._subtree_inorder(self.left(p)):
            yield other
    yield p 
    if self.right(p) is not None:
        for other in self._subtree_inorder(self.right(p)):
            yield other

LinkedBinaryTree._subtree_inorder = _subtree_inorder
LinkedBinaryTree.inorder = inorder

# Expected output: 4, 2, 5, 6, 3, 7
print('--INORDER TRAVERSAL--')
for p in tree.inorder():
    print(p.element())

--PREORDER TRAVERSAL--
1
2
4
5
3
6
7


--POSTORDER TRAVERSAL--
4
5
2
6
7
3
1


--INORDER TRAVERSAL--
4
2
5
1
6
3
7


**Exercicio 4:**<br>
Escreva um algoritmo eficiente para verificar se duas arvores binarias sao identicas ou nao. Duas arvores binarias sao identicas se tiverem a mesma estrutura e o mesmo conteudo.

In [ ]:
# Algorithm to check if two ArrayBinaryTrees are equal.
def are_abt_equal(t1: ArrayBinaryTree, t2:ArrayBinaryTree):
    if len(t1) is not len(t2):
        return False
    for i in range(0, len(t2)):
        if t1[i] is not t2[i]:
            return False
    return True

# Algorithm to check if two LinkedBinaryTrees are equal
def are_lbt_equal(t1: LinkedBinaryTree, t2: LinkedBinaryTree):
    return are_identical_subtrees(t1.root(), t2.root())

def are_identical_subtrees(p1, p2):
    if p1 is None and p2 is None:
        return True
    if p1 is None or p2 is None:
        return False
    if p1.element() != p2.element():
        return False
    
    left_identical = are_identical_subtrees(p1.left(), p2.left())
    right_identical = are_identical_subtrees(p1.right(), p2.right())

    return left_identical and right_identical

t1 = LinkedBinaryTree()
t1._add_root(1)
t2 = LinkedBinaryTree()
t2._add_root(1)
are_lbt_equal(t1, t2)

AttributeError: 'Position' object has no attribute 'left'